In [4]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, Point, mapping
import os

In [2]:
admin2 = gpd.read_file('gadm36_ETH_shp/gadm36_ETH_3.shp')
admin2['Gadm Name 2'] = admin2['NAME_2']
admin2['Gadm Name 1'] = admin2['NAME_1']

In [3]:
pts3 = admin2.centroid.unary_union

from shapely.ops import nearest_points
# unary union of the gpd2 geomtries 

def near(point, pts=pts3):
     # find the nearest point and return the corresponding Place value
    nearest = admin2.centroid == nearest_points(point, pts)[1]
    return admin2[nearest]['Gadm Name 2'].get_values()[0]


def near_gadm1(geom, pts=pts3):
    if geom.type == Polygon:
        point = geom.centroid
    else:
        point = geom
     # find the nearest point and return the corresponding Place value
    nearest = admin2.centroid == nearest_points(point, pts)[1]
    return admin2[nearest]['Gadm Name 1'].get_values()[0]

#### Read in DSSAT files

Baseline - The baseline outputs to compare to:

ERAIN - Modified rainfall from 25% to 200% (025 - 200)

PLANW - Modified planting date start window with -30, -15, +15, and +30
days

FEN_TOT - Modified fertilizer total adjustment in addition to the baseline amount
by 15-60 kg/ha

In [7]:
results = [i for i in os.listdir('eth_sensitivity/') if '.csv' in i]

In [15]:
results

['ETH_MZ_BH660_erain_025_cells.csv',
 'ETH_MZ_BH660_erain_150_cells.csv',
 'ETH_MZ_BH660_erain_075_cells.csv',
 'ETH_MZ_BH660_fen_tot15_cells.csv',
 'ETH_MZ_BH660_planw_+15_0_cells.csv',
 'ETH_MZ_BH660_planw_-30_0_cells.csv',
 'ETH_MZ_BH660_baseline_cells.csv',
 'ETH_MZ_BH660_erain_200_cells.csv',
 'ETH_MZ_BH660_erain_175_cells.csv',
 'ETH_MZ_BH660_planw_+30_0_cells.csv',
 'ETH_MZ_BH660_planw_-15_0_cells.csv',
 'ETH_MZ_BH660_erain_050_cells.csv',
 'ETH_MZ_BH660_erain_125_cells.csv',
 'ETH_MZ_BH660_fen_tot60_cells.csv',
 'ETH_MZ_BH660_fen_tot30_cells.csv']

In [31]:
df = pd.DataFrame()

for r in results:
    s = r.split('660_')[1].split('_cells')[0]
    if s == 'baseline':
        scenario = 'Baseline'
        p = 0
    if 'erain' in s:
        p = int(s.split('_')[1])
        scenario = "Rainfall"
    if 'planw' in s:
        p = int(s.split('planw_')[1].split('_')[0])
        scenario = 'Planting date'
    if 'fen_tot' in s:
        p = int(s.split('tot')[1])
        scenario = 'Fertilizer'
    print(r, s, scenario, p)
    df_ = pd.read_csv(f'eth_sensitivity/{r}')
    df_['Perturbation'] = p
    df_['Scenario'] = scenario
    df = df.append(df_)

ETH_MZ_BH660_erain_025_cells.csv erain_025 Rainfall 25
ETH_MZ_BH660_erain_150_cells.csv erain_150 Rainfall 150
ETH_MZ_BH660_erain_075_cells.csv erain_075 Rainfall 75
ETH_MZ_BH660_fen_tot15_cells.csv fen_tot15 Fertilizer 15
ETH_MZ_BH660_planw_+15_0_cells.csv planw_+15_0 Planting date 15
ETH_MZ_BH660_planw_-30_0_cells.csv planw_-30_0 Planting date -30
ETH_MZ_BH660_baseline_cells.csv baseline Baseline 0
ETH_MZ_BH660_erain_200_cells.csv erain_200 Rainfall 200
ETH_MZ_BH660_erain_175_cells.csv erain_175 Rainfall 175
ETH_MZ_BH660_planw_+30_0_cells.csv planw_+30_0 Planting date 30
ETH_MZ_BH660_planw_-15_0_cells.csv planw_-15_0 Planting date -15
ETH_MZ_BH660_erain_050_cells.csv erain_050 Rainfall 50
ETH_MZ_BH660_erain_125_cells.csv erain_125 Rainfall 125
ETH_MZ_BH660_fen_tot60_cells.csv fen_tot60 Fertilizer 60
ETH_MZ_BH660_fen_tot30_cells.csv fen_tot30 Fertilizer 30


In [32]:
df.groupby(['Scenario','Perturbation']).HWAM.count()

Scenario       Perturbation
Baseline        0              5924
Fertilizer      15             5924
                30             5924
                60             5924
Planting date  -30             5924
               -15             5924
                15             5924
                30             5924
Rainfall        25             5924
                50             5924
                75             5924
                125            5924
                150            5924
                175            5924
                200            5924
Name: HWAM, dtype: int64

In [34]:
df['Geometry'] = df.apply(lambda row: Point(row.LONGITUDE, row.LATITUDE), axis=1)

In [46]:
geom = df[['LATITUDE','LONGITUDE']].drop_duplicates()
geom['Geometry'] = geom.apply(lambda row: Point(row.LONGITUDE, row.LATITUDE), axis=1)
geom['Gadm Name 1'] = geom.Geometry.apply(lambda x: near_gadm1(x))
geom['Gadm Name 2'] = geom.Geometry.apply(lambda x: near(x))

In [92]:
df = df.join(geom.set_index(['LATITUDE','LONGITUDE']), how='left', on=['LATITUDE','LONGITUDE'], rsuffix='g_')

In [98]:
baseline = df[df['Scenario']=='Baseline'][['LATITUDE','LONGITUDE','HWAM']]

In [100]:
df = df.join(baseline.set_index(['LATITUDE','LONGITUDE']), how='left', on=['LATITUDE','LONGITUDE'], rsuffix='_baseline')

In [101]:
df['Yield Delta from Baseline'] = df['HWAM'] - df['HWAM_baseline']

In [102]:
df.to_csv("DSSAT-Sensitivity.csv", index=False)